In [ ]:
import numpy as np
from numpy import mean
from numpy import isnan
from numpy import asarray
from numpy import polyfit
from numpy import loadtxt
from scipy.stats import pearsonr
from sklearn.dummy import DummyClassifier
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
DATASET = loadtxt('dataset.csv', delimiter=",")

In [ ]:
NUMBER_OF_PARAMETER = "INSERT_HERE"
X = DATASET[:,0:NUMBER_OF_PARAMETER]
Y = DATASET[:,NUMBER_OF_PARAMETER]

In [ ]:
def import_models():
	models = list()
	models.append(DummyClassifier(strategy="most_frequent"))
	models.append(DummyClassifier(strategy="stratified"))
	models.append(xgb.XGBClassifier())
	models.append(LogisticRegression())
	models.append(KNeighborsClassifier())
	models.append(DecisionTreeClassifier())
	models.append(SVC())
	models.append(AdaBoostClassifier())
	models.append(RandomForestClassifier())
	models.append(GradientBoostingClassifier())
	models.append(MLPClassifier()) 
	return models

In [ ]:
def evaluate_model(cv, model):
    
    model_accuracy = 0
            
    scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average='weighted', zero_division=0),
           'recall' : make_scorer(recall_score, average='weighted', zero_division=0), 
           'f1_score' : make_scorer(f1_score, average='weighted', zero_division=0)}
    
    if "XGB" in str(model):
        le = LabelEncoder()
        Y_transformed = le.fit_transform(Y)
        scores = cross_validate(model, X, Y_transformed, scoring=scoring, cv=cv, n_jobs=2)

    else:
        scores = cross_validate(model, X, Y, scoring=scoring, cv=cv, n_jobs=2)
         
    for k,v in scores.items():
        
        if k == "test_accuracy":
            print('accuracy=%.3f' % (mean(v)))
            model_accuracy = mean(v)
            
        if k == "test_precision":
            print('precision=%.3f' % (mean(v)))
            
        if k == "test_recall":
            print('recall=%.3f' % (mean(v)))
            
        if k == "test_f1_score":
            print('f1=%.3f' % (mean(v)))
            
    return model_accuracy


In [ ]:
cv = KFold(n_splits=10, shuffle=True, random_state=1)
models = import_models()

In [ ]:
cv_results = list()
for model in models:
	cv_mean = evaluate_model(cv, model)
	if isnan(cv_mean):
		continue
	print('>%s: cv=%.3f' % (type(model).__name__, cv_mean))